# Assitants API with GPT-4 Turbo model

We will crate a simple math tutor assistant that is able to solve math equations and create python code to get the answer.

### 1. Install the OpenAI Python SDK

Ensure the OpenAI Python SDK is installed or upgraded:

In [1]:
%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 956.3 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 5.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.5/162.5 kB 11.4 MB/s eta 0:00:00

[notice] A new

### 2. Create a Math Assistant:

Use the API to create an assistant with specific instructions tailored for math tutoring. In the instructions, specify that the assistant should provide detailed explanations and, if applicable, Python code snippets for solving math problems.

In [20]:
import openai
import time

openai.api_key = "<your OpenAI API key>"

math_assistant = openai.beta.assistants.create(
    name="Math Tutor Assistant Turbo",
    instructions="Explain math problems step-by-step and provide Python code snippets for solutions when appropriate.",
    model="gpt-4-turbo-preview"  # Example model
)


In [21]:
import time

### 3. Using Threads and Runs for Interactions:
Create a thread and run for each interaction. In this scenario, the user's math problem or question is sent as a message, and the assistant responds with an explanation and code.

In [22]:
# Create a thread for the conversation
thread = openai.beta.threads.create()
thread_id = thread.id

# Function to wait for the execution to complete
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = openai.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

# Function to submit a math question and get a response
def ask_math_question(question):
    # Add the user's question to the thread
    openai.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=question
    )

    # Run the assistant
    run = openai.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=math_assistant.id,
    )

    # Wait for the response
    run = wait_on_run(run, thread)
    response = openai.beta.threads.messages.list(thread_id=thread_id, order="asc")
    
    if response.data:
        return response.data[-1].content[0].text
    else:
        return "No response received."

# Example usage
response = ask_math_question("How do I calculate the derivative of x^2 + 3x + 5?")
print(response)


Text(annotations=[], value='To calculate the derivative of the function \\(f(x) = x^2 + 3x + 5\\), you\'ll follow the basic rules of differentiation. The derivative gives you how the function changes at any given point. For polynomials like this, you can apply the power rule, which states that if you have a term \\(ax^n\\), its derivative is \\(n \\cdot ax^{n-1}\\).\n\nGiven the function:\n\\[f(x) = x^2 + 3x + 5\\]\n\nLet\'s differentiate it term by term:\n\n1. **First term (\\(x^2\\)):** Apply the power rule, where \\(a = 1\\) and \\(n = 2\\).\n   - The derivative of \\(x^2\\) is \\(2 \\cdot x^{2-1} = 2x\\).\n2. **Second term (\\(3x\\)):** Here, \\(a = 3\\) and \\(n = 1\\).\n   - The derivative of \\(3x\\) is \\(1 \\cdot 3x^{1-1} = 3\\cdot x^0 = 3\\). Note that \\(x^0 = 1\\), so you\'re just left with the constant \\(3\\).\n3. **Third term (\\(5\\)):** This is a constant term. The derivative of any constant is \\(0\\), since constants do not change and hence their "rate of change" is 

### 4. Handling the Response:
The ask_math_question function sends a math question to the assistant and waits for a response. The response will include an explanation and, when relevant, a Python code snippet demonstrating the solution.
Remember to replace "YOUR_OPENAI_API_KEY" with your actual OpenAI API key. The code assumes that the assistant can handle math problems and generate Python code as part of its response. Adjust the model and instructions as necessary for your specific use case.

In [23]:
from IPython.display import display, Markdown, Latex

display(Markdown(response.value))

To calculate the derivative of the function \(f(x) = x^2 + 3x + 5\), you'll follow the basic rules of differentiation. The derivative gives you how the function changes at any given point. For polynomials like this, you can apply the power rule, which states that if you have a term \(ax^n\), its derivative is \(n \cdot ax^{n-1}\).

Given the function:
\[f(x) = x^2 + 3x + 5\]

Let's differentiate it term by term:

1. **First term (\(x^2\)):** Apply the power rule, where \(a = 1\) and \(n = 2\).
   - The derivative of \(x^2\) is \(2 \cdot x^{2-1} = 2x\).
2. **Second term (\(3x\)):** Here, \(a = 3\) and \(n = 1\).
   - The derivative of \(3x\) is \(1 \cdot 3x^{1-1} = 3\cdot x^0 = 3\). Note that \(x^0 = 1\), so you're just left with the constant \(3\).
3. **Third term (\(5\)):** This is a constant term. The derivative of any constant is \(0\), since constants do not change and hence their "rate of change" is zero.

Putting it all together, the derivative of \(f(x) = x^2 + 3x + 5\) is:
\[f'(x) = 2x + 3 + 0\]
\[f'(x) = 2x + 3\]

Hence, the derivative is \(2x + 3\).

### Python Code
To calculate this derivative using Python, you could use the sympy library, which is designed for symbolic mathematics, including differentiation. Here's a small snippet to do just that:

```python
from sympy import symbols, diff

# declare the symbol
x = symbols('x')

# define the function
f_x = x**2 + 3*x + 5

# calculate the derivative
f_prime = diff(f_x, x)

print(f_prime)  # This will print the derivative
```

This code snippet will print the derivative of the function \(f(x) = x^2 + 3x + 5\), which is \(2x + 3\), same as we calculated manually.

# 

# Multimodal Capabilities and TTS API 

We will ask GPT 4 if there is a mouse in the following image:

![alternative text](https://cdn.fantasticpestscontrol.com.au/wp-content/uploads/2017/04/house_mouse.jpg)

### 1. Open the image
We will use the base64 library to create an b64 encoded image object.

In [29]:
import base64
import openai
import os

# Set OpenAI Key
openai.api_key = "<your OpenAI API key>"

# Updated file path to a JPEG image
image_path = "/Users/<your user>/Documents/mouse_picture.jpeg"

# Read and encode the image in base64
with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

### 2. Create the request
We create to request for the completion call targeting the gpt-4-vision-preview model.

In [31]:
# Craft the prompt for GPT
prompt_messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Here is an image, is there a mouse in the image?"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                }
            }
        ]
    }
]

# Send a request to GPT
params = {
    "model": "gpt-4-vision-preview",
    "messages": prompt_messages,
    "max_tokens": 4096,
}

### 3. Execute
We execute the call and check the results.

In [32]:
result = openai.chat.completions.create(**params)
print(result.choices[0].message.content)

Yes, there is a mouse in the image. It appears to be standing on some leaves or vegetation.
